In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn

pd.set_option('display.max_columns', None)

train_data = pd.read_csv(r"/kaggle/input/santander-customer-transaction-prediction/train.csv")
test_data = pd.read_csv(r"/kaggle/input/santander-customer-transaction-prediction/test.csv")

train_X = train_data.drop(columns=['ID_code','target'])
train_y = train_data['target']
test_X = test_data.drop(columns=['ID_code'])

In [ ]:
print(train_X.shape)
print(train_y.shape)
print(test_X.shape)

In [ ]:
sns.countplot(train_y)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

scale = np.logspace(0,2,num=10)
grid = [{0:1, 1:weight} for weight in scale]
grid = {"class_weight": grid }
l2_model = LogisticRegression(penalty='l2', random_state=25, solver='sag')
grid_search = GridSearchCV(l2_model,grid,scoring="roc_auc", n_jobs=-1, refit=True, verbose=2)

print(grid)
print("-----------------Start Fitting Weighted Logistic Regression Model With L-2 regularization-----------------")
grid_search.fit(train_X, train_y)

In [ ]:
train_pred = grid_search.predict(train_X)

In [ ]:
## compute the confusion matrix for binary classification problem
def confusion_matrix(y, y_pred, threshold=None):
    assert(len(y) == len(y_pred))
    
    if threshold is not None:
        y_pred = y_pred >= threshold
    
    
    #  TP  |  FN
    #  ---------
    #  FP  |  TN
    
    mat = np.zeros((2,2))
    
    for i in np.arange(len(y)):
        if y_pred[i] == y[i]:
            if y[i] == 1:
                ## TP
                mat[0,0] = mat[0,0] + 1
            if y[i] == 0:
                ## TN
                mat[1,1] = mat[1,1] + 1
        else:
            if y[i] == 1:
                ## FN
                mat[0,1] = mat[0,1] + 1
            if y[i] == 0:
                ## FP:
                mat[1,0] = mat[1,0] + 1
    
    return mat

In [ ]:
confusion_mat = confusion_matrix(train_y, train_pred)
confusion_mat

In [ ]:
tpr = confusion_mat[0,0] / (confusion_mat[0,0] + confusion_mat[1,1])
fpr = confusion_mat[1,0] / (confusion_mat[1,1] + confusion_mat[1,0])
precision = confusion_mat[0,0] / (confusion_mat[0,0] + confusion_mat[1,0])
print("True positive rate: " + str(tpr))
print("False positive rate: " + str(fpr))
print("precision" + str(precision))

In [ ]:
### submission
### https://www.kaggle.com/dansbecker/submitting-from-a-kernel

test_pred = grid_search.predict(test_X)
my_submission = pd.DataFrame({'ID_code': test_data.ID_code, 'target': test_pred})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)